In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/Amphibians.csv',sep=';')

In [4]:
df.shape

(190, 23)

In [5]:
df

,Integer,Categorical,Numerical,Numerical.1,Categorical.1,Categorical.2,Categorical.3,Categorical.4,Categorical.5,Categorical.6,...,Ordinal.1,Categorical.8,Categorical.9,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7
0,ID,Motorway,SR,NR,TR,VR,SUR1,SUR2,SUR3,UR,...,BR,MR,CR,Green frogs,Brown frogs,Common toad,Fire-bellied toad,Tree frog,Common newt,Great crested newt
1,1,A1,600,1,1,4,6,2,10,0,...,0,0,1,0,0,0,0,0,0,0
2,2,A1,700,1,5,1,10,6,10,3,...,1,0,1,0,1,1,0,0,1,0
3,3,A1,200,1,5,1,10,6,10,3,...,1,0,1,0,1,1,0,0,1,0
4,4,A1,300,1,5,0,6,10,2,3,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,185,S52,2300,1,12,3,2,2,1,0,...,1,0,1,0,1,0,0,0,0,0
186,186,S52,300,1,14,2,7,10,2,0,...,5,0,1,1,1,1,1,0,1,0
187,187,S52,500,1,1,4,1,10,2,0,...,5,0,1,1,1,1,1,0,1,0
188,188,S52,300,1,12,3,2,1,6,0,...,0,0,1,0,1,1,0,0,0,0


In [6]:
bdf = BaseDatasetTransform(df,target=['Label 1','Label 2','Label 3','Label 4','Label 5','Label 6','Label 7'])

In [7]:
df,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['Label 1', 'Label 2', 'Label 3', 'Label 4', 'Label 5', 'Label 6', 'Label 7'] нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Integer', 'Categorical', 'Numerical', 'Numerical.1', 'Categorical.1',
       'Categorical.2', 'Categorical.3', 'Categorical.4', 'Categorical.5',
       'Categorical.6', 'Categorical.7', 'Numerical.2', 'Ordinal', 'Ordinal.1',
       'Categorical.8', 'Categorical.9'],
      dtype='object')

Числовые колонки:
Index([], dtype='object')


In [8]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям:
Количество значений комбинаций целевых переменных по категориям:
0_1_0_0_0_0_0                                                                                     23
0_1_1_0_0_0_0                                                                                     19
1_1_1_1_1_1_0                                                                                     12
1_1_1_0_0_0_0                                                                                     12
1_1_1_1_1_1_1                                                                                     10
1_1_1_0_1_0_0                                                                                      9
1_1_1_1_0_1_0                                                                                      7
0_1_1_0_1_0_0                                                                                      7
1_1_1_1_1_0_0                                                            

Разделение датасета выполнено успешно


In [9]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    #objective='binary',  # Для бинарной классификации
    #metric='binary_error'  # Метрика для бинарной задачи
)

7


In [10]:
metrics = [ 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Integer',
                                                                  'Categorical',
                                                                  'Numerical',
                                                                  'Numerical.1',
                                                                  'Categorical.1',
                                                                  'Categorical.2',
                                                                  'Categorical.3',
                                                                  'Categorical.4',
                                                                  'Categorical.5',
                                                                  'Categorical.6',
                                                                  'Categorical.7',
                                                                  'Numerical.2',
                                                                  'Ordinal',
                                                                  'Ordinal.1',
                                                                  'Categorical.8',
                                                                  'Categorical.9'],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [11]:
model_train.predict(X_test)

array([['0', '1', '1', '0', '0', '0', '0'],
       ['0', '1', '1', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '1', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '1', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '1', '0', '0'],
       ['0', '1', '1', '0', '0', '0', '0'],
       ['0', '1', '1', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '1', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '1', '0', '0'],
       ['0', '1', '1', '0', '0', '0', '0'],
       ['0', '1', '1', '0', '0', '0', '0'],
       ['0', '1', '1', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['0', '0', '1', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '1',

In [12]:
y_test

,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7
175,0,1,1,1,1,1,1
180,1,1,1,0,1,1,0
111,1,1,1,1,1,1,1
65,1,1,1,0,0,0,0
101,1,1,1,0,0,0,0
15,0,1,1,0,0,0,0
9,0,1,1,0,0,0,0
16,1,1,1,1,1,0,0
141,1,1,1,1,1,1,0
124,0,1,0,0,0,0,0


In [13]:
evaluator.evaluate_to_dataframe(X_test, y_test)

True
Evaluation results: {'precision': 0.49153765979035674, 'recall': 0.534793168592994, 'f1': 0.48006522918086264}


,Metric,Score
0,precision,0.491538
1,recall,0.534793
2,f1,0.480065
